<a href="https://colab.research.google.com/github/mahasalmanc/home_sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=1f986857fca5a1294828d462729c76ad83cd47405145e8fa1fee914bb4c0ed95
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
spark.sparkContext.addFile(url)
home_sales_data = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
home_sales_data.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_data.createOrReplaceTempView('home')


In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("select ROUND(AVG(price),2), YEAR(date) from home where bedrooms == 4 group by YEAR(date) ORDER BY YEAR(date)").show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|            300263.7|      2019|
|           298353.78|      2020|
|           301819.44|      2021|
|           296363.88|      2022|
+--------------------+----------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("select ROUND(AVG(price),2), date_built from home where bedrooms == 3 AND bathrooms == 3 group by date_built ORDER BY date_built").show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           292859.62|      2010|
|           291117.47|      2011|
|           293683.19|      2012|
|           295962.27|      2013|
|           290852.27|      2014|
|            288770.3|      2015|
|           290555.07|      2016|
|           292676.79|      2017|
+--------------------+----------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("select ROUND(AVG(price),2), date_built from home where bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000 group by date_built ORDER BY date_built").show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           285010.22|      2010|
|           276553.81|      2011|
|           307539.97|      2012|
|           303676.79|      2013|
|           298264.72|      2014|
|           297609.97|      2015|
|            293965.1|      2016|
|           280317.58|      2017|
+--------------------+----------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price), 2) from home group by view having ROUND(AVG(price),2) >= 350000")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.08467268943786621 seconds ---


In [14]:
spark.sql("SELECT view, ROUND(AVG(price), 2) from home group by view having ROUND(AVG(price),2) >= 350000")


DataFrame[view: int, round(avg(price), 2): double]

In [15]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home")

DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home')

True

In [17]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()


spark.sql("SELECT view, ROUND(AVG(price), 2) from home group by view having ROUND(AVG(price),2) >= 350000")


print("--- %s seconds ---" % (time.time() - start_time))


--- 0.036077260971069336 seconds ---


In [18]:
print("the cached runtime was 0.04859542846 seconds shorter than original")

the cached runtime was 0.04859542846 seconds shorter than original


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_data.write.parquet('home_parquet', mode='overwrite')

In [20]:
# 11. Read the parquet formatted data.
parquet_home_sales = spark.read.parquet('home_parquet')

In [22]:
# 12. Create a temporary table for the parquet data.
parquet_home_sales.createOrReplaceTempView('parquet_temp')

In [23]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price), 2) from parquet_temp group by view having ROUND(AVG(price),2) >= 350000")


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.031844139099121094 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home")

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home')

False